In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.linear_model import LogisticRegression
from category_encoders import HashingEncoder, CountEncoder
from xgboost import XGBClassifier
from scipy.stats import skew, kurtosis


class DataProcessor:
    def __init__(self):
        self.encoders = {}
        self.scaler = StandardScaler()

    @staticmethod
    def _convert_time_columns(data):
        data['start_time'] = pd.to_datetime(
            data['start_time'], format='%Y%m%d%H%M%S')
        data['end_time'] = pd.to_datetime(
            data['end_time'], format='%Y%m%d%H%M%S')
        return data

    @staticmethod
    def _extract_time_features(data):
        data['start_hour'] = data['start_time'].dt.hour
        data['start_dayofweek'] = data['start_time'].dt.dayofweek
        data['is_weekend'] = data['start_dayofweek'].apply(
            lambda x: 1 if x >= 5 else 0)
        data['is_working_hour'] = data['start_hour'].apply(
            lambda x: 1 if 9 <= x <= 18 else 0)
        return data

    def _encode_categorical_features(self, data, encoding_config):
        for feature, encoding_method in encoding_config.items():
            if encoding_method == 'onehot':
                encoder = OneHotEncoder(
                    sparse_output=False, handle_unknown='ignore')
                encoded = encoder.fit_transform(data[[feature]])
                encoded_df = pd.DataFrame(
                    encoded, columns=[f"{feature}_{cat}" for cat in encoder.categories_[0]])
                data = pd.concat([data, encoded_df], axis=1)
                data.drop(columns=[feature], inplace=True)
            elif encoding_method == 'label':
                encoder = OrdinalEncoder(
                    handle_unknown='use_encoded_value', unknown_value=-1)
                encoded = encoder.fit_transform(data[[feature]])
                data[feature] = encoded
            elif encoding_method == 'hash':
                encoder = HashingEncoder()
                encoded = encoder.fit_transform(data[[feature]])
                data = pd.concat([data, encoded], axis=1)
            elif encoding_method == 'count':
                encoder = CountEncoder()
                encoded = encoder.fit_transform(data[[feature]])
                data[feature] = encoded
            elif encoding_method == 'labelcount':
                encoder = CountEncoder(normalize=True)
                encoded = encoder.fit_transform(data[[feature]])
                data[feature] = encoded
            else:
                raise ValueError(f"Unknown encoding method: {encoding_method}")
            self.encoders[feature] = encoder
        return data

    @staticmethod
    def _statistical_features(df, feature):
        return df.groupby('msisdn')[feature].agg([
            'sum', 
            'mean', 
            'max', 
            'min', 
            'std', 
            'var', 
            'median', 
            'nunique', 
            'size', 
            'count',
            # ('skew', lambda x: skew(x) if len(x) > 1 else np.nan),
            # ('kurt', lambda x: kurtosis(x) if len(x) > 1 else np.nan),
            ('quantile_25', lambda x: x.quantile(0.25)),
            ('quantile_75', lambda x: x.quantile(0.75)),
            ('mode', lambda x: x.mode().iloc[0]
             if not x.mode().empty else np.nan)
        ]).add_prefix(f'{feature}_')

    @staticmethod
    def _aggregate_numerical_features(data, numerical_features):
        user_aggregated_data = pd.DataFrame()
        for feature in numerical_features:
            feature_stats = DataProcessor._statistical_features(data, feature)
            if user_aggregated_data.empty:
                user_aggregated_data = feature_stats
            else:
                user_aggregated_data = user_aggregated_data.join(
                    feature_stats, how='outer')
        return user_aggregated_data

    @staticmethod
    def _aggregate_categorical_frequencies(data, categorical_features, user_aggregated_data):
        categorical_features = [
            'call_event', 'roam_type', 'long_type1', 'ismultimedia',  'phone1_type', 'phone2_type', 'is_weekend', 'is_working_hour',
        ]
        for feature in categorical_features:
            frequency = data.groupby(
                ['msisdn', feature]).size().unstack(fill_value=0)
            normalized_frequency = frequency.div(frequency.sum(axis=1), axis=0)
            normalized_frequency.columns = [
                f"{feature}_{col}_freq" for col in normalized_frequency.columns]
            user_aggregated_data = user_aggregated_data.join(
                normalized_frequency, how='left')
        return user_aggregated_data

    @staticmethod
    def _aggregate_differential_features(data):
        data['call_duration_diff'] = data.groupby(
            'msisdn')['call_duration'].diff().fillna(0)
        data['call_duration_diff2'] = data.groupby(
            'msisdn')['call_duration'].diff(2).fillna(0)

        diff_agg_funcs = {
            'call_duration_diff': ['mean', 'std'],
            'call_duration_diff2': ['mean', 'std']
        }

        diff_aggregated_data = data.groupby('msisdn').agg(diff_agg_funcs)
        diff_aggregated_data.columns = [
            '_'.join(map(str, col)).strip() for col in diff_aggregated_data.columns.values]

        return diff_aggregated_data

    @staticmethod
    def _binary_operations(user_aggregated_data):
        if 'cfee_sum' in user_aggregated_data.columns and 'lfee_sum' in user_aggregated_data.columns:
            user_aggregated_data['cfee_lfee_sum'] = user_aggregated_data['cfee_sum'] + \
                user_aggregated_data['lfee_sum']
            user_aggregated_data['cfee_lfee_diff'] = user_aggregated_data['cfee_sum'] - \
                user_aggregated_data['lfee_sum']
            user_aggregated_data['cfee_lfee_prod'] = user_aggregated_data['cfee_sum'] * \
                user_aggregated_data['lfee_sum']
            user_aggregated_data['cfee_lfee_ratio'] = user_aggregated_data['cfee_sum'] / (
                user_aggregated_data['lfee_sum'] + 1e-6)
        return user_aggregated_data

    def preprocess_and_aggregate(self, data, label_data=None, is_validation=False, fit_columns=None, encoding_config=None):
        data = self._convert_time_columns(data)
        data = self._extract_time_features(data)

        # 疑似类型标志
        suspect_types = {3, 5, 6, 9, 11, 12, 17}
        data['is_suspect'] = data['phone1_type'].apply(
            lambda x: 1 if x in suspect_types else 0)

        categorical_features = [
            'call_event', 'roam_type', 'long_type1', 'ismultimedia', 'home_area_code',
            'visit_area_code', 'called_home_code', 'called_code', 'a_serv_type',
            'a_product_id', 'phone1_type', 'phone2_type', 'phone1_loc_city',
            'phone1_loc_province', 'phone2_loc_city', 'phone2_loc_province',
            'is_weekend', 'is_working_hour'
        ]
        data = self._encode_categorical_features(data, encoding_config)

        numerical_features = ['call_duration', 'cfee',
                              'lfee', 'start_hour', 'start_dayofweek']
        user_aggregated_data = self._aggregate_numerical_features(
            data, numerical_features)
        user_aggregated_data = self._aggregate_categorical_frequencies(
            data, categorical_features, user_aggregated_data)
        diff_aggregated_data = self._aggregate_differential_features(data)
        user_aggregated_data = user_aggregated_data.join(
            diff_aggregated_data, how='left')

        user_aggregated_data = self._binary_operations(user_aggregated_data)
        user_aggregated_data.fillna(0, inplace=True)
        user_aggregated_data.reset_index(inplace=True)

        if not is_validation and label_data is not None:
            user_aggregated_data = user_aggregated_data.merge(
                label_data, on='msisdn', how='left')

        if not is_validation:
            numerical_features = [
                col for col in user_aggregated_data.columns if col not in ['msisdn', 'is_sa']]
            user_aggregated_data[numerical_features] = self.scaler.fit_transform(
                user_aggregated_data[numerical_features])
            return user_aggregated_data, numerical_features, self.scaler
        else:
            user_aggregated_data[fit_columns] = self.scaler.transform(
                user_aggregated_data[fit_columns])
            return user_aggregated_data


def feature_selection(train_data, label_column='is_sa', k=20):
    X = train_data.drop(columns=['msisdn', label_column])
    y = train_data[label_column]

    # XGBoost feature importance
    xgb_model = XGBClassifier(eval_metric='logloss', n_jobs=-1)
    xgb_model.fit(X, y)
    xgb_feature_scores = pd.Series(
        xgb_model.feature_importances_, index=X.columns)
    xgb_selected_features = xgb_feature_scores.nlargest(k).index.tolist()

    # Mutual information
    mutual_info_scores = mutual_info_classif(X, y)
    mutual_info_series = pd.Series(mutual_info_scores, index=X.columns)
    mutual_info_selected_features = mutual_info_series.nlargest(
        k).index.tolist()

    # Recursive feature elimination
    rfe_model = LogisticRegression(max_iter=1000)
    rfe = RFE(rfe_model, n_features_to_select=k)
    rfe.fit(X, y)
    rfe_selected_features = X.columns[rfe.support_].tolist()

    # Combine selected features from all methods
    combined_features = list(set(
        xgb_selected_features + mutual_info_selected_features + rfe_selected_features))

    return combined_features


# Encoding configuration with Count Encoding for location features
encoding_config = {
    'call_event': 'label',          # 通话类型：label encoding
    'other_party': 'label',          # 对端标识号2：label encoding
    'ismultimedia': 'label',        # 视频呼叫标志：label encoding
    'home_area_code': 'count',      # 归属地区号：count encoding
    'visit_area_code': 'count',     # 到访地区号：count encoding
    'called_home_code': 'count',    # 对端归属区号：count encoding
    'called_code': 'count',         # 对端到访区号：count encoding
    'a_serv_type': 'onehot',        # 用户业务类型：one hot encoding
    'long_type1': 'label',          # 长途类型1：label encoding
    'roam_type': 'label',           # 漫游类型：label encoding
    'a_product_id': 'count',        # 产品编码：count encoding
    'phone1_type': 'label',         # 标识号1类型：label encoding
    'phone2_type': 'label',         # 对端标识号2类型：label encoding
    'phone1_loc_city': 'count',     # 标识号1的通话所在地：count encoding
    'phone1_loc_province': 'count',  # 标识号1的通话所在省：count encoding
    'phone2_loc_city': 'count',     # 对端标识号2的通话所在地：count encoding
    'phone2_loc_province': 'count',  # 对端标识号2的通话所在省：count encoding
    'is_weekend': 'label',          # 是否周末：label encoding
    'is_working_hour': 'label'      # 是否工作时间：label encoding
}

In [ ]:
# 读取数据
train_set_res = pd.read_csv(
    '/home/hwxu/Projects/Competition/Telecom/Input/raw/train.csv', low_memory=False)
train_set_ans = pd.read_csv(
    '/home/hwxu/Projects/Competition/Telecom/Input/raw/labels.csv', low_memory=False)
validation_set_res = pd.read_csv(
    '/home/hwxu/Projects/Competition/Telecom/Input/raw/val.csv', low_memory=False)

In [ ]:
# 实例化数据处理器
data_processor = DataProcessor()

# 处理训练集
train_data, fit_columns, scaler = data_processor.preprocess_and_aggregate(
    train_set_res, train_set_ans, is_validation=False, encoding_config=encoding_config)

# 选择特征
selected_features = feature_selection(train_data, k=50)
train_data = train_data[['msisdn'] + selected_features + ['is_sa']]

# 处理验证集
validation_data = data_processor.preprocess_and_aggregate(
    validation_set_res, is_validation=True, fit_columns=fit_columns, encoding_config=encoding_config)

# 选择特征（根据训练集选择的特征）
validation_data = validation_data[['msisdn'] + selected_features]

# 输出处理后的训练集和验证集
train_data.to_csv(
    '/home/hwxu/Projects/Competition/Telecom/Input/processed/train.csv', index=False)
validation_data.to_csv(
    '/home/hwxu/Projects/Competition/Telecom/Input/processed/val.csv', index=False)

In [ ]:
train_data.shape, validation_data.shape